In [1]:
!pip install pyngrok transformers torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import os

# Create necessary folders
os.makedirs('templates', exist_ok=True)
os.makedirs('uploads', exist_ok=True)

# Create index.html
index_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Pet Breed Detection</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body {
            background: #f8f9fa;
        }
        .card {
            margin-top: 100px;
        }
    </style>
</head>
<body>
    <div class="container d-flex justify-content-center">
        <div class="card shadow p-4" style="width: 100%; max-width: 500px;">
            <h3 class="text-center mb-4">🐾 Pet Breed Detection</h3>
            <form action="/predict" method="post" enctype="multipart/form-data">
                <div class="mb-3">
                    <label for="image" class="form-label">Upload a Pet Image</label>
                    <input class="form-control" type="file" name="image" accept="image/*" required>
                </div>
                <button type="submit" class="btn btn-primary w-100">Predict Breed</button>
            </form>
        </div>
    </div>
</body>
</html>
"""

# Create result.html
result_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Prediction Result</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container text-center mt-5">
        <h2>Prediction Result</h2>
        <img src="{{ url_for('uploaded_file', filename=filename) }}" alt="Uploaded Image" class="img-thumbnail mt-3" style="max-width: 300px;">
        <div class="alert alert-success mt-3" role="alert" style="font-size: 1.5rem;">
            🐶 Predicted Breed: <strong>{{ breed }}</strong><br>
            🔍 Confidence: <strong>{{ confidence }}%</strong>
        </div>
        <a href="/" class="btn btn-secondary">Try Another Image</a>
    </div>
</body>
</html>
"""


with open("templates/index.html", "w") as f:
    f.write(index_html)

with open("templates/result.html", "w") as f:
    f.write(result_html)


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from flask import Flask, request, render_template
import torch
import json
from PIL import Image
from transformers import ViTImageProcessor, ViTForImageClassification
import os
from pyngrok import ngrok
from flask import send_from_directory



# Open a public URL for the Flask app
NGROK_AUTH_TOKEN = "2xUiujC8qsCJE6mgBQLpVDl6xwB_6Ng1T1RPhtvrPb9aNEEW8"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
# Constants
IMG_SIZE = (224, 224)
UPLOAD_FOLDER = 'uploads'

# Flask setup
app = Flask(__name__, template_folder='templates')
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
#run_with_ngrok(app)

# Load model and processor
model_path = "/content/drive/MyDrive/saved_model"
processor = ViTImageProcessor.from_pretrained(model_path)
model = ViTForImageClassification.from_pretrained(model_path)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load label map
with open("/content/drive/MyDrive/saved_model/trainlabel_mapping.json") as f:
    label_map = json.load(f)
idx_to_class = {v: k for k, v in label_map.items()}

def preprocess_image(img_path):
    image = Image.open(img_path).convert("RGB").resize(IMG_SIZE)
    inputs = processor(images=image, return_tensors="pt")
    return inputs["pixel_values"]

def predict_breed(img_path):
    pixel_values = preprocess_image(img_path).to(device)
    with torch.no_grad():
        outputs = model(pixel_values)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
        predicted_idx = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][predicted_idx].item() * 100  # Convert to percentage
    return idx_to_class[predicted_idx], confidence


@app.route('/')
def home():
    return render_template('index.html')

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)
@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return "<h2>Error: No image uploaded.</h2>"

    file = request.files['image']
    if file.filename == '':
        return "<h2>Error: Empty filename.</h2>"

    filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(filepath)
    try:
        breed, confidence = predict_breed(filepath)
        return render_template('result.html', breed=breed, confidence=round(confidence, 2), filename=file.filename)
    except Exception as e:
        return f"<h2>Error during prediction: {str(e)}</h2>"

# Open a tunnel on port 5000 for the Flask app

public_url = ngrok.connect(5000)
print(" * ngrok tunnel URL:", public_url)
app.run(port=5000)

 * ngrok tunnel URL: NgrokTunnel: "https://200e-34-80-180-90.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/May/2025 13:35:28] "GET /robots.txt HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 13:35:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 13:35:31] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 13:35:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 13:35:44] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 13:35:57] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 13:35:57] "GET /uploads/inbound6270622912756987088.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 13:36:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 13:36:06] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/May/2025 13:36:42] "